In [1]:
from google.colab import files
files.upload()

import os
os.makedirs("/root/.kaggle", exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json



Saving kaggle.json to kaggle.json


In [2]:
# ========== 3. download and unzip the dataset ==========
!kaggle datasets download -d paramaggarwal/fashion-product-images-dataset

Dataset URL: https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-dataset
License(s): MIT


In [3]:
# Check if the file exists before attempting to open it
import zipfile
import os
if os.path.exists("fashion-product-images-dataset.zip"):
    with zipfile.ZipFile("fashion-product-images-dataset.zip", 'r') as zip_ref:
        zip_ref.extractall("fashion_data")
else:
    print("Error: 'fashion-product-images-dataset.zip' not found. Please ensure the download was successful.")



In [4]:
import pandas as pd
df = pd.read_csv("/content/fashion_data/fashion-dataset/fashion-dataset/styles.csv", on_bad_lines='skip')

In [5]:
# Get unique masterCategory labels
unique_labels = df['masterCategory'].dropna().unique()

# Count of unique labels
num_labels = len(unique_labels)

print(f"Number of unique masterCategory labels: {num_labels}")
print("Labels:", unique_labels)

Number of unique masterCategory labels: 7
Labels: ['Apparel' 'Accessories' 'Footwear' 'Personal Care' 'Free Items'
 'Sporting Goods' 'Home']


In [6]:
import os
import shutil
from tqdm import tqdm

# Create a root dataset folder
os.makedirs("fashion_data_mas", exist_ok=True)

# Create subfolders for each masterCategory label
for label in df['masterCategory'].unique():
    os.makedirs(os.path.join("fashion_data_mas", label), exist_ok=True)

# Move images into label folders
source_dir = "/content/fashion_data/fashion-dataset/images"  # Folder from the dataset where original images are stored
for idx, row in tqdm(df.iterrows(), total=len(df)):
    img_name = '{}.jpg'.format(row['id'])
    label = row['masterCategory']
    src = os.path.join(source_dir, img_name)
    dst = os.path.join("fashion_data_mas", label, img_name)
    if os.path.exists(src):
        shutil.copy(src, dst)

100%|██████████| 44424/44424 [00:41<00:00, 1074.00it/s]


In [8]:
import torch
import torch.nn as nn
import timm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load EfficientNet with 7 output classes
model = timm.create_model("efficientnet_b0", pretrained=True, num_classes=7)
model = model.to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

In [9]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


full_dataset = datasets.ImageFolder("fashion_data_mas", transform=transform)


total_size = len(full_dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size


train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)


train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [10]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [11]:
import torch
from sklearn.metrics import f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=10, class_names=None):
    best_f1 = 0.0

    for epoch in range(epochs):
        # ---- Training ----
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)

        # ---- Validation ----
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)
        f1 = f1_score(all_labels, all_preds, average='weighted')

        # ---- Save Best Model ----
        if f1 > best_f1:
            best_f1 = f1
            torch.save(model.state_dict(), "best_model.pth")
            print(f"✅ Best model updated at Epoch {epoch+1} | F1: {f1:.4f}")

        # ---- Confusion Matrix ----
        cm = confusion_matrix(all_labels, all_preds)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=class_names if class_names else np.unique(all_labels),
                    yticklabels=class_names if class_names else np.unique(all_labels))
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
        plt.title(f'Confusion Matrix - Epoch {epoch+1}')
        plt.tight_layout()
        plt.savefig(f'confusion_matrix_epoch_{epoch+1}.png')
        plt.close()

        # ---- Logging ----
        print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | "
              f"Val Loss: {avg_val_loss:.4f} | F1 Score: {f1:.4f}")


In [12]:
train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    device=device,
    epochs=10
)


✅ Best model updated at Epoch 1 | F1: 0.9894
Epoch 1 | Train Loss: 0.1086 | Val Loss: 0.1865 | F1 Score: 0.9894
Epoch 2 | Train Loss: 0.0283 | Val Loss: 0.2782 | F1 Score: 0.9886
✅ Best model updated at Epoch 3 | F1: 0.9907
Epoch 3 | Train Loss: 0.0212 | Val Loss: 0.0615 | F1 Score: 0.9907
✅ Best model updated at Epoch 4 | F1: 0.9927
Epoch 4 | Train Loss: 0.0178 | Val Loss: 0.3277 | F1 Score: 0.9927
✅ Best model updated at Epoch 5 | F1: 0.9928
Epoch 5 | Train Loss: 0.0156 | Val Loss: 0.0502 | F1 Score: 0.9928
Epoch 6 | Train Loss: 0.0112 | Val Loss: 0.0662 | F1 Score: 0.9888
✅ Best model updated at Epoch 7 | F1: 0.9934
Epoch 7 | Train Loss: 0.0110 | Val Loss: 0.0538 | F1 Score: 0.9934
Epoch 8 | Train Loss: 0.0072 | Val Loss: 0.0566 | F1 Score: 0.9930
Epoch 9 | Train Loss: 0.0094 | Val Loss: 0.0649 | F1 Score: 0.9908
Epoch 10 | Train Loss: 0.0071 | Val Loss: 0.0584 | F1 Score: 0.9932


In [13]:
from sklearn.metrics import accuracy_score, f1_score
import torch.nn.functional as F
import copy

In [15]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

acc = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='macro')
print(f"\n📊 Test Accuracy: {acc:.4f}, F1 (macro): {f1:.4f}")


📊 Test Accuracy: 0.9952, F1 (macro): 0.6647


In [16]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

acc = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='weighted')
print(f"\n📊 Test Accuracy: {acc:.4f}, F1 (weighted): {f1:.4f}")


📊 Test Accuracy: 0.9952, F1 (weighted): 0.9943


In [ ]:
torch.save(model.state_dict(), "/content/efficientnet_fashion_mas_big.pt")